In [ ]:
!pip install datasets transformers

In [ ]:
import tensorflow
from datasets import load_dataset
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from transformers import AutoImageProcessor, TFViTForImageClassification, TFAutoModelForImageClassification

In [ ]:
dataset = load_dataset('cats_vs_dogs')['train'].train_test_split(
    test_size=0.1, stratify_by_column='labels', seed=42,
)['test'].train_test_split(
    test_size=0.1, stratify_by_column='labels', seed=42,
)

Generating train split:   0%|          | 0/23410 [00:00<?, ? examples/s]

In [ ]:
def process_image(sample):
    image_features = processor(sample['image'], return_tensors='tf')
    return {'pixel_values': tensorflow.squeeze(image_features['pixel_values'])}

In [ ]:
processor = AutoImageProcessor.from_pretrained('google/vit-base-patch16-224',)
model = TFAutoModelForImageClassification.from_pretrained('google/vit-base-patch16-224')

model.classifier = Dense(2)

All PyTorch model weights were used when initializing TFViTForImageClassification.

All the weights of TFViTForImageClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTForImageClassification for predictions without further training.


In [ ]:
dataset = dataset.map(process_image)

Map:   0%|          | 0/2106 [00:00<?, ? examples/s]

Map:   0%|          | 0/235 [00:00<?, ? examples/s]

In [ ]:
tf_train_dataset = dataset['train'].to_tf_dataset(
    columns=['pixel_values'],
    label_cols=['labels'],
    shuffle=True,
    batch_size=32,
)
tf_test_dataset = dataset['test'].to_tf_dataset(
    columns=['pixel_values'],
    label_cols=['labels'],
    shuffle=False,
    batch_size=32,
)

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:400: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [ ]:
model.compile(
    optimizer='adam',
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)
model.fit(tf_train_dataset, validation_data=tf_test_dataset, epochs=3)

Epoch 1/3
66/66 [==============================] - 569s 7s/step - loss: 0.9822 - accuracy: 0.5095 - val_loss: 0.6753 - val_accuracy: 0.5532
Epoch 2/3
66/66 [==============================] - 507s 8s/step - loss: 0.7695 - accuracy: 0.5176 - val_loss: 0.7276 - val_accuracy: 0.4979
Epoch 3/3
66/66 [==============================] - 512s 8s/step - loss: 0.7087 - accuracy: 0.5252 - val_loss: 0.6904 - val_accuracy: 0.5404


In [ ]:
!pip install tensorflow_model_optimization

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 4.4 MB/s eta 0:00:00


In [ ]:
model.summary()

Model: "tf_vi_t_for_image_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vit (TFViTMainLayer)        multiple                  85798656  
                                                                 
 dense (Dense)               multiple                  1538      
                                                                 
Total params: 85800194 (327.30 MB)
Trainable params: 85800194 (327.30 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
_, baseline_model_accuracy = model.evaluate(
    tf_test_dataset, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

Baseline test accuracy: 0.5404255390167236


In [ ]:
import tempfile
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
checkpoint_path = '/content/drive/MyDrive/models_weights/baseline/'

In [ ]:
model.save_weights(checkpoint_path)

In [ ]:
import numpy as np

In [ ]:
features = model(inputs['pixel_values']).last_hidden_state

NameError: ignored

In [ ]:
model.layers[1]

In [ ]:
import tensorflow_model_optimization as tfmot

cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

clustering_params = {
  'number_of_clusters': 16,
  'cluster_centroids_init': CentroidInitialization.LINEAR
}

# Cluster a whole model
clustered_model_2 = cluster_weights(model, **clustering_params)

# Use smaller learning rate for fine-tuning clustered model
opt = tf.keras.optimizers.Adam(learning_rate=1e-5)

clustered_model_2.compile(
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  optimizer=opt,
  metrics=['accuracy'])

clustered_model_2.summary()
# на это этапе возникает ошибка, пробуем с отдельным слоем

ValueError: ignored

In [ ]:
import tensorflow_model_optimization as tfmot
from transformers import AutoImageProcessor, TFAutoModelForImageClassification


# Определяем параметры для кластеризации
clustering_params = {
  'number_of_clusters': 16,
  'cluster_centroids_init': tfmot.clustering.keras.CentroidInitialization.LINEAR
}

# Выбераем слой для кластеризации (Dense)
layer_to_cluster = model.layers[1]  # Это пример, замените индекс на нужный вам слой

# Кластеризуем выбранный слой
clustered_layer = tfmot.clustering.keras.cluster_weights(layer_to_cluster_1, **clustering_params)

# Создаем новую модель с кластеризованным слоем
clustered_model = tf.keras.models.Sequential([
    model.layers[0], clustered_layer
])


All PyTorch model weights were used when initializing TFViTForImageClassification.

All the weights of TFViTForImageClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTForImageClassification for predictions without further training.


In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=1e-5)

clustered_model.compile(
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  optimizer=opt,
  metrics=['accuracy'])


In [ ]:
# на обучении модели тоже возникает ошибка
clustered_model.fit(tf_train_dataset, validation_data=tf_test_dataset, epochs=3)

Epoch 1/3


AttributeError: ignored

In [ ]:
_, clustered_model_accuracy = clustered_model.evaluate(
  test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Clustered test accuracy:', clustered_model_accuracy)

In [ ]:
final_model = tfmot.clustering.keras.strip_clustering(clustered_model)

_, clustered_keras_file = tempfile.mkstemp('.h5')
print('Saving clustered model to: ', clustered_keras_file)
tf.keras.models.save_model(final_model, clustered_keras_file,
                           include_optimizer=False)